In [1]:
import numpy as np
from scipy.integrate import quad
import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator

In [2]:
#Se definen algunos parámetricos cosmológicos estándares
neff0=3.04 # número efectivo de especies relativista
h=.6736 #H0/100 km/s/Mpc
om_bar0=0.051 #parámetro de densidad bariónico a z=0
om_dm0=0.264 #parámetro de densidad de materia oscura a z=0
om_lam0=0.68 #parámetro de densidad de constante cosmológica a z=0
om_rad0=9.237e-05 #parámetro de densidad de radiación a z=0

print(om_bar0+om_dm0)
c=299792 #km/s #velocidad de la luz
kmtoMpc=3.2408e-20 #factor de conversión de km a Mpc
H0=h*100
H0s=(H0)*kmtoMpc
c_over_H0=c*kmtoMpc/H0s #c/H0
c_over_H0

0.315


4450.593824228029

Modelos cosmologicos

In [3]:
# Se define la clase para la Cosmología estándar LCDM
class CosmologiaLCDM:
    def __init__(self,h=.6736,odm0=0.264, or0=9.237e-05, ob0=0.051):
        #Aquí se definen los parámetros del modelo cosmológicos como propiedades de la clase
        self._h= h
        self._Omega_dm0 = odm0 
        self._Omega_rad0 = or0
        self._Omega_bar0 = ob0    
    
    #Atributos de la clase
    def Ezmodel(self, z=0):
        #This function returns the dimensionless Friedmann equation E(z)=H(z)/H0 for LCDM
        z1=1.0+z
        
        z13=np.power(z1,3.)
        
        z14=np.power(z1,4.)
        
        OMAT=(self._Omega_dm0+self._Omega_bar0)*z13 #Matter density (dark+baryionic) parameter as function of z
        
        ORAD=self._Omega_rad0*z14 #radiation density parameter as function of z
        
        ODE=1.0- self._Omega_dm0 - self._Omega_bar0-self._Omega_rad0 #Flatness condition
        
        x=np.sqrt(OMAT +ORAD + ODE)
        
        return x
    
    def fmodel(self,z):
        #This function returns the inverse of the  dimensionless Friedmann equation
        return 1.0/self.Ezmodel(z)
    
    def Dc(self,z):
        #This function returns the transverse comoving distance for a flat Universe
        return c_over_H0*quad(lambda x:self.fmodel(x),0,z)[0]

    def DA(self,z):
        #This function returns the angular diameter distance to any source at redshift z
        return (1/(1+z))*self.Dc(z)

    def DA12(self,z1,z2):
        #"This function returns the angular diameter distance between two objects"
        return (c_over_H0/(1+z2))*quad(lambda x:self.fmodel(x),z1,z2)[0]

    def DL(self,z):
         #"This function returns the luminosity distance at redshift z"
        return (1+z)*self.Dc(z)


In [4]:
cos1=CosmologiaLCDM() #En esta línea se define una variable cos1 con la Cosmología LCDM
Dc1=cos1.Dc(1) # Aquí acceso al atributo distancia comóvil y evaluo a z=1
print(Dc1)

3402.949023150786


In [5]:
# se define la clase para la Cosmología Cardassiana
class CosmologiaCardassian:
    def __init__(self,h=.6736,odm0=0.264, or0=9.237e-05, ob0=0.051, nfid=-0.2):
        self._h= h
        self._Omega_dm0 = odm0 
        self._Omega_rad0 = or0
        self._Omega_bar0 = ob0 
        self._n=nfid
        
    def Ezmodel(self, z=0):
        
        z1=1.0+z
        
        z13=np.power(z1,3.)
        
        z14=np.power(z1,4.)
        
        OMAT0=(self._Omega_dm0+self._Omega_bar0) #Matter density (dark+baryionic) parameter at z=0
        
        OMAT=OMAT0*z13 #Matter density (dark+baryionic) parameter as function of z
        
        ORAD=self._Omega_rad0*z14 #radiation density parameter as function of z
        
        ODE=1.0-OMAT0-self._Omega_rad0
        
        OMSTD=OMAT+ORAD
        
        x=OMSTD/(OMAT0+self._Omega_rad0)
        
        fde=np.power(x,self._n)
        
        y=np.sqrt(OMSTD +fde )
        
        return y

    def fmodel(self,z):
        #"This function returns the inverse of the  dimensionless Friedmann equation"
        return 1.0/self.Ezmodel(z)
    
    def Dc(self,z):
        #"This function returns the transverse comoving distance for a flat Universe"
        return c_over_H0*quad(lambda x:self.fmodel(x),0,z)[0]

    def DA(self,z):
        #"This function returns the angular diameter distance to any source at redshift z"
        return (1/(1+z))*self.Dc(z)

    def DA12(self,z1,z2):
        #"This function returns the angular diameter distance between two objects"
        return (c_over_H0/(1+z2))*quad(lambda x:self.fmodel(x),z1,z2)[0]

    def DL(self,z):
         #"This function returns the luminosity distance at redshift z"
        return (1+z)*self.Dc(z)
    def Dm(z,h,om_bar0,om_dm0,om_rad0):#Comoving distance transverse 
    if om_k>0:
        D_m=c_over_H0*(om_k)**(-0.5)*np.sinh((om_k)**(0.5)*Dc(z,h,om_bar0,om_dm0,om_rad0)/c_over_H0)
    elif om_k==0:
        D_m=Dc(z,h,om_bar0,om_dm0,om_rad0)      
    else:
        D_m=c_over_H0*(abs(om_k))**(-0.5)*np.sinh((abs(om_k))**(0.5)*Dc(z,h,om_bar0,om_dm0,om_rad0)/c_over_H0)
    return D_m
    def tL(z,h,om_bar0,om_dm0,om_rad0): #lookback time tL
    th=1/(100*h) #tiempo de hubble
    om_lamb=(1.0-om_dm0-om_rad0-om_bar0)
    if type(z)==int or type(z)==float:
        Dc=quad(lambda x:f(x,h,om_bar0,om_dm0,om_rad0)/(x+1), 0, z)[0]
    else:
        Dc=np.array([])
        for i in range(len(z)):
            Dc=np.append(Dc,quad(lambda x:f(x,h,om_bar0,om_dm0,om_rad0)/(x+1), 0, z[i])[0])                
    return th*Dc

In [6]:
cos2=CosmologiaCardassian() #En esta línea se define una variable cos1 con la Cosmología LCDM
Dc2=cos2.Dc(1) # Aquí acceso al atributo distancia comóvil y evaluo a z=1
print(Dc2)

3249.350090929249
